In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
from ydata_profiling import ProfileReport
import os
import warnings
import sys
import pandas as pd

sys.path.append("../")
from pipeline import data
from pipeline.config import CONF
from pipeline.data import plots

# To suppress all warnings
warnings.filterwarnings("ignore")

# black is a code formatter (see https://github.com/psf/black).
# It will automatically format the code you write in the cells imposing consistent Python style.
%load_ext jupyter_black
# matplotlib style file
# Template for style file: https://matplotlib.org/stable/tutorials/introductory/customizing.html#customizing-with-style-sheets
plt.style.use("../matplotlib_style.txt")
pd.set_option("display.max_columns", None)  # Show all columns
pd.set_option("display.expand_frame_repr", False)  # Prevent wrapping

### Raw data inspection

In [ ]:
# Load raw data
(
    Installed_Capacity_Germany,
    Prices_Europe,
    Realised_Supply_Germany,
    Realised_Demand_Germany,
    Weather_Data_Germany,
) = data.load_data(CONF=CONF, data_type="raw")

In [ ]:
# Generate profile reports
data.save_data_inspection(
    Installed_Capacity_Germany=Installed_Capacity_Germany,
    Prices_Europe=Prices_Europe,
    Realised_Supply_Germany=Realised_Supply_Germany,
    Realised_Demand_Germany=Realised_Demand_Germany,
    Weather_Data_Germany=Weather_Data_Germany,
    CONF=CONF,
    data_type="raw",
)

### Raw data processing

In [ ]:
data.save_data(
    Installed_Capacity_Germany=data.process_na_values(Installed_Capacity_Germany, CONF),
    Prices_Europe=data.process_na_values(Prices_Europe, CONF),
    Realised_Supply_Germany=data.process_na_values(Realised_Supply_Germany, CONF),
    Realised_Demand_Germany=data.process_na_values(Realised_Demand_Germany, CONF),
    Weather_Data_Germany=data.process_na_values(Weather_Data_Germany, CONF),
    CONF=CONF,
    data_type="preprocessed",
)

### Processed data inspection

In [ ]:
# Load processed data
(
    Installed_Capacity_Germany,
    Prices_Europe,
    Realised_Supply_Germany,
    Realised_Demand_Germany,
    Weather_Data_Germany,
) = data.load_data(CONF, data_type="preprocessed")

In [ ]:
# Generate profile reports
data.save_data_inspection(
    Installed_Capacity_Germany=Installed_Capacity_Germany,
    Prices_Europe=Prices_Europe,
    Realised_Supply_Germany=Realised_Supply_Germany,
    Realised_Demand_Germany=Realised_Demand_Germany,
    Weather_Data_Germany=Weather_Data_Germany,
    CONF=CONF,
    data_type="preprocessed",
)

### Plots processed data

In [ ]:
plots.plot_df(Installed_Capacity_Germany, "Installed_Capacity_Germany", CONF)
plots.plot_df(Prices_Europe, "Prices_Europe", CONF)
plots.plot_df(Realised_Supply_Germany, "Realised_Supply_Germany", CONF)
plots.plot_df(Realised_Demand_Germany, "Realised_Demand_Germany", CONF)